# genetic algorithm 遗传算法

## 1.Application

1. 寻找函数全局的最优解（最大值或者最小值）
2. 机器学习中优化模型的超参数
3. 旅行商问题：寻找最短路径
4. 背包问题： 在限制下使得总价值（目标函数）最大化
5. 任务分配问题

## 2. Algorithm

1. 核心思想 ： 模拟种群进化，其本质是通过**选择、交叉、变异**操作，像最优解进化
2. 基本概念 ： 
  * Individual 个体 ： 一个候选解
  * population 种群 ： 一组个体，代表当前搜索的解空间
  * Fitness Function 适应度函数 : 衡量个体优劣的标准
3. 工作流程 ： 
  * 对问题编码 ： 1.二进制编码,适用于离散问题 2.实数编码，适用于连续问题 3. 排列编码 ：适用于组合优化问题
  * 初始化种群 ： 1.随机初始化，随机生成满足变量范围的解 2.启发式初始化 ；利用先验知识生成初始解
  ``` def initialize_population(pop_size,bounds)
      # 随机生成满足变量范围内的解
        return np.random.uniform(bounds[0],bounds[1],pop_size)
  ```
  * 计算适应度函数 ： 
  ```    def fitness_function(x):
       # 例如：求函数 f(x) = x * sin(10π * x) + 2.0 的最大值
            return x * np.sin(10 * np.pi * x) + 2.0
  ```
  * 选择操作 ： 根据适应度选择优秀个体
  ```    def select_parents(population, fitness):
       # 轮盘赌选择法
            probs  = fitness / fitness.sum()
            selected_indices  =  np.random.choice(len(population),size=len(population),p=probs)
            return population[selected_indices]
  ```
    假如适应度函数为负数，那么可以使用适应度函数平移的方法
  ``` 
      shifted_fitness = fitness - np.min(fitness)
  ```
  *交叉操作 ： 产生新一代个体
  ```    def crossover(parents):
            offspring = np.zeros(len(parents))
            for i in range(0,len(parents),2):
              beta = np.random.random()
              offspring[i] = parents[i] * beta + parents[i+1]*(1-beta)
              offspring[i+1] = parents[i+1] * beta + parents[i]*(1-beta)
            return offspring
  ```
  *变异操作 ： 维持种群多样性
  ```    def mutate(offspring,mutate_rate,bounds):
          # mask 是用于记录每一个个体是否将发生变异数组
          mask = np.random.random(len(offspring)) < mutate_rate
          #生成变异值
          mutation = np.random.normal(0,0.1,len(offspring))
          offspring[mask] += mutation[mask]
          offspring = np.clip(offspring, bounds[0], bounds[1])
          return offspring
  ```
             


In [8]:
import numpy as np

mask = np.random.random(10) < 0.3

mutation = np.random.normal(0, 0.1, 10)

mutation

array([ 0.05913532,  0.01746663,  0.04205566, -0.09679154, -0.10405432,
       -0.05181892, -0.11917878,  0.00022595,  0.00602761,  0.10980543])